## Постановка задачи
У нас появился запрос из отдела продаж и маркетинга. Как вы знаете «МегаФон» предлагает обширный набор различных услуг своим абонентам. При этом разным пользователям интересны разные услуги. Поэтому необходимо построить  алгоритм, который **для каждой пары пользователь-услуга определит вероятность подключения услуги**.

### Исходные данные

#### наборы данных
- **features.csv** (22Г): id, buy_time, <feature_list>
нормализованный анонимизированный набор признаков, характеризующий профиль потребления абонента. Эти данные привязаны к определенному времени, поскольку профиль абонента может меняться с течением времени.
- **data_train.csv** (27М): id, vas_id, buy_time, target
информация об отклике абонентов на предложение подключения одной из услуг. Каждому пользователю может быть сделано несколько предложений в разное время, каждое из которых он может или принять, или отклонить.
- **data_test.csv**: id, vas_id, buy_time
тестовый набор

#### переменные
   - **target** - целевая переменная, где 1 означает подключение услуги, 0 - абонент не подключил услугу соответственно.
   - **buy_time** - время покупки, представлено в формате timestamp, для работы с этим столбцом понадобится функция datetime.fromtimestamp из модуля datetime.
   - **id** - идентификатор абонента
   - **vas_id** - подключаемая услуга




### Анализ данных

1. считаем data_train и data_test и проверим на разнообразие услуг и абонентов
2. Восстановим наборы абонентов и услуг
3. обработаем features.csv отфильруем только то что есть в data_train и data_test!

![Схема](data_schema.png)

In [1]:
import pandas as pd

Считаем исходные данные

In [16]:
data_train=pd.read_csv('data_train.csv')
data_train.head(3)

,Unnamed: 0,id,vas_id,buy_time,target
0,0,540968,8.0,1537131600,0.0
1,1,1454121,4.0,1531688400,0.0
2,2,2458816,1.0,1534107600,0.0


In [18]:
data_test=pd.read_csv('data_test.csv')
data_test.head(3)

,Unnamed: 0,id,vas_id,buy_time
0,0,3130519,2.0,1548018000
1,1,2000860,4.0,1548018000
2,2,1099444,2.0,1546808400


Объединим наборы для восстановления id абонентов

In [26]:
data_all = pd.concat([data_train.drop(columns='target'), data_test])
data_all.head(3)

,Unnamed: 0,id,vas_id,buy_time
0,0,540968,8.0,1537131600
1,1,1454121,4.0,1531688400
2,2,2458816,1.0,1534107600


In [27]:
data_all.shape[0], data_all.id.nunique(), data_all.vas_id.nunique()

(902884, 872577, 8)

> нам необходимо отфильтровать features.csv по 872 тыс абонентов

Создадим для экспериментов срез файла features.csv - первые 5 тыс. записей

In [38]:
num_lines = 5000
with open("features_lite.csv", "x") as lite:
    with open("features.csv") as f:
       for line in f:
           lite.write(line)
           num_lines -= 1
           if num_lines%100 == 0:
               print(num_lines, line[0:20])
           if num_lines <= 0:
               break

4900 98	2402038	154559880
4800 198	2813843	15462036
4700 298	3554903	15328980
4600 398	3661489	15419700
4500 498	3748859	15310836
4400 598	3863771	15468084
4300 698	3963168	15322932
4200 798	4047744	15377364
4100 898	4129298	15310836
4000 998	4201230	15322932
3900 1098	4270438	1538946
3800 1198	4344118	1535922
3700 1298	50258	154257480
3600 1398	133220	15389460
3500 1498	216934	15310836
3400 1598	293264	15377364
3300 1698	366781	15455988
3200 1798	439178	15365268
3100 1898	568256	15425748
3000 1998	879298	15310836
2900 2098	923040	15449940
2800 2198	942906	15449940
2700 2298	961432	15462036
2600 2398	981936	15377364
2500 2498	1004744	1541365
2400 2598	1040382	1532293
2300 2698	1071139	1545598
2200 2798	1089994	1533502
2100 2898	1111412	1535317
2000 2998	1128739	1544994
1900 3098	1146979	1534107
1800 3198	1165797	1546203
1700 3298	1186322	1536526
1600 3398	1241864	1544389
1500 3498	1257553	1535317
1400 3598	1278188	1544389
1300 3698	1297428	1543784
1200 3798	1317219	1542574
1100 3898	13

In [40]:
features = pd.read_csv('features_lite.csv', sep="\t")
features.head(3)

,Unnamed: 0,id,buy_time,0,1,2,3,4,5,6,...,243,244,245,246,247,248,249,250,251,252
0,0,2013026,1531688400,18.910029,46.980888,4.969214,-1.386798,3.791754,-14.01179,-16.08618,...,-977.373846,-613.770792,-25.996269,-37.630448,-301.747724,-25.832889,-0.694428,-12.175933,-0.45614,0.0
1,1,2014722,1539550800,36.690029,152.400888,448.069214,563.833202,463.841754,568.99821,-16.08618,...,-891.373846,-544.770792,-20.996269,48.369552,80.252276,-13.832889,-0.694428,-1.175933,-0.45614,0.0
2,2,2015199,1545598800,-67.019971,157.050888,-63.180786,178.103202,-68.598246,156.99821,3.51382,...,-977.373846,-613.770792,-12.996269,-37.630448,10829.252276,-25.832889,-0.694428,-12.175933,-0.45614,0.0


In [42]:
features.shape

(4999, 256)

In [44]:
merge=features.id.isin(data_all.id)
features1 = features[merge]

In [45]:
features1

,Unnamed: 0,id,buy_time,0,1,2,3,4,5,6,...,243,244,245,246,247,248,249,250,251,252
13,13,2046132,1534712400,300.820029,1599.480888,286.879214,1585.013202,281.461754,1563.90821,-16.08618,...,-977.373846,-613.770792,-25.996269,-35.630448,-295.747724,-17.832889,-0.694428,-4.175933,-0.45614,0.0
16,16,2050810,1540760400,-86.209971,91.820888,-84.480786,110.333202,-89.898246,89.22821,-16.08618,...,-977.373846,-613.770792,-23.996269,190.369552,-286.747724,-25.832889,-0.694428,-12.175933,-0.45614,0.0
19,19,2070757,1540760400,-96.799971,-408.179112,-110.740786,-460.786798,-114.038246,-479.77179,-16.08618,...,-925.373846,-561.770792,-21.996269,-37.630448,-151.747724,-24.832889,0.305572,-12.175933,-0.45614,1.0
20,20,2071522,1544994000,-94.939971,-363.699112,-108.880786,-411.226798,-114.298246,-432.33179,-16.08618,...,-977.373846,-613.770792,-25.996269,-37.630448,-306.747724,-25.832889,-0.694428,-12.175933,-0.45614,0.0
22,22,2075318,1533502800,-75.639971,669.690888,-89.580786,732.343202,-94.998246,736.65821,-16.08618,...,-501.373846,-242.770792,-25.996269,-37.630448,-167.747724,-14.832889,2.305572,-4.175933,-0.45614,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4962,4962,1826618,1532293200,-1.109971,-25.419112,6.919214,14.923202,1.501754,-6.18179,-16.08618,...,-977.373846,-613.770792,-14.996269,-35.630448,-304.747724,-25.832889,-0.694428,-12.175933,-0.45614,1.0
4969,4969,1840249,1544389200,-96.799971,-88.699112,-110.740786,-141.306798,-113.618246,-159.87179,-16.08618,...,-969.373846,-606.770792,-19.996269,-33.630448,-250.747724,-5.832889,13.305572,-12.175933,-0.45614,1.0
4974,4974,1843245,1533502800,-34.509971,-142.499112,-48.450786,-195.106798,-53.868246,-165.36179,-16.08618,...,-976.373846,-613.770792,-25.996269,-32.630448,-225.747724,-15.832889,-0.694428,-2.175933,-0.45614,0.0
4983,4983,1852915,1535317200,85.380029,692.640888,79.919214,665.463202,74.501754,778.25821,-16.08618,...,-872.373846,-557.770792,-6.996269,-35.630448,-210.747724,13.167111,-0.694428,26.824067,-0.45614,1.0


In [47]:
data_all.id.min(), data_all.id.max()

(2, 4362720)

In [49]:
features1.id.value_counts().sum()

946